# Criar Sistema para Detecção Facial

In [ ]:
import imutils
import numpy as np
import cv2
from google.colab.patches import cv2_imshow
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

Usar o Darknet para detectar faces em uma imagem de teste.

In [ ]:
def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)

  # get photo data
  data = eval_js('takePhoto({})'.format(quality))
  # get OpenCV format image
  img = js_to_image(data)
  # grayscale img
  gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
  print(gray.shape)
  # get face bounding box coordinates using Haar Cascade
  faces = face_cascade.detectMultiScale(gray)
  # draw face bounding box on image
  for (x,y,w,h) in faces:
      img = cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
  # save image
  cv2.imwrite(filename, img)

  return filename

In [ ]:
# Instalar o CUDA 10.1
!apt-get update
!apt-get install -y --no-install-recommends cuda-10-1
!ln -s /usr/local/cuda-10.1 /usr/local/cuda
!apt-get install -y --no-install-recommends cuda-drivers

# Reiniciar o ambiente para que as mudanças tenham efeito
import os
os.kill(os.getpid(), 9)


Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,084 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,858 kB]
Get:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com/ubuntu 

Use o Darknet para detectar faces na imagem capturada pela webcam:

In [ ]:
# Realizar a detecção de faces na imagem capturada
!./darknet detect cfg/yolov3.cfg yolov3.weights {filename}

# Exibir a imagem com as detecções
Image(filename='predictions.jpg')



Classificação das Faces Detectadas

In [ ]:
# Função para extrair as coordenadas das detecções do Darknet
def extract_faces(image_path, darknet_output_path='predictions.jpg'):
    # Carregar a imagem original
    original_image = cv2.imread(image_path)

    # Lendo as coordenadas das detecções do arquivo de resultados do Darknet
    faces = []
    with open("predictions.txt", "r") as f:
        for line in f.readlines():
            class_name, confidence, x, y, w, h = line.strip().split()
            x, y, w, h = int(x), int(y), int(w), int(h)
            face = original_image[y:y+h, x:x+w]
            faces.append(face)

    return faces

# Extrair faces da imagem capturada
faces = extract_faces(filename)

# Carregar o modelo pré-treinado VGG16
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions
import numpy as np

model = VGG16(weights='imagenet')

# Função para classificar as faces detectadas
def classify_faces(faces):
    for i, face in enumerate(faces):
        face = cv2.resize(face, (224, 224))
        x = image.img_to_array(face)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)

        preds = model.predict(x)
        print(f"Face {i+1}: {decode_predictions(preds, top=3)[0]}")

# Classificar as faces detectadas
classify_faces(faces)
